In [31]:
from transformer_hyperparam import get_compiled_transformer
import os
import pandas as pd
from copy import deepcopy
import numpy as np
import random
from tensorflow import keras

class ASLDataGenerator(keras.utils.Sequence):

    def __init__(self, dataset_path: str, max_phrase_length: int, max_sign_length: int, batch_size: int, train: bool = True):
        super().__init__()
        self.ds_path = dataset_path
        self.files = os.listdir(dataset_path)
        self.max_phrase_length = max_phrase_length
        self.max_sign_length = max_sign_length
        self.unwanted_columns = ['sequence_id', 'frame', 'participant_id', 'phrase']
        self.train = train
        self.batch_size = batch_size

    def __len__(self):
        return len(self.files)//self.batch_size

    def shuffle(self):
        random.shuffle(self.files)

    def on_epoch_end(self):
        if self.train:
            self.shuffle()

    def __getitem__(self, idx: int):
        batch_signs = []
        batch_contexts = []
        batch_phrases = []
        for i in range(idx, idx+self.batch_size):
            df_phrase = pd.read_pickle(self.ds_path + '/' + self.files[idx])
            phrase = df_phrase.iloc[0].phrase
            if len(phrase) > self.max_phrase_length:
                print('Houston we have a problem!')

            while len(phrase) < self.max_phrase_length:
                phrase.append(59)
            context = deepcopy(phrase)
            context.insert(0, 60)
            context = context[:-1]

            df_phrase = df_phrase.drop(self.unwanted_columns, axis=1)
            signs = df_phrase.to_numpy(copy=True)

            if signs.shape[0] > self.max_sign_length:
                print('Hooooouuuuusteeeeeen!')
            signs = np.pad(signs, [(0, self.max_sign_length-signs.shape[0]), (0,0)], 'edge')
            batch_signs.append(signs)
            batch_contexts.append(context)
            batch_phrases.append(phrase)

        batch_signs = np.array(batch_signs)
        batch_contexts = np.array(batch_contexts)
        batch_phrases = np.array(batch_phrases)

        return [batch_signs, batch_contexts], batch_phrases

In [32]:
MAX_PHRASE_LENGTH = 100
MAX_SIGN_LENGTH = 900
BATCH_SIZE=32

data_gen_train = ASLDataGenerator('./dataset complete/preprocessed_files_data_generator/train_ones', MAX_PHRASE_LENGTH, MAX_SIGN_LENGTH, batch_size=BATCH_SIZE)
data_gen_train.shuffle()

data_gen_test = ASLDataGenerator('./dataset complete/preprocessed_files_data_generator/test_ones', MAX_PHRASE_LENGTH, MAX_SIGN_LENGTH, train=False, batch_size=BATCH_SIZE)

data_gen_val = ASLDataGenerator('./dataset complete/preprocessed_files_data_generator/val_ones', MAX_PHRASE_LENGTH, MAX_SIGN_LENGTH, train=False, batch_size=BATCH_SIZE)

In [33]:
import optuna
import json

epochs = 5

def objective(trial):
    #starting from:
    # d_model = trial.suggest_int('d_model', 20, 256)
    #num_layers = trial.suggest_int('num_layers', 1, 10)
    #num_heads = trial.suggest_int('num_heads', 2, 10)
    #ff_dim = trial.suggest_int('ff_dim', 32, 1024)
    #dropout_rate = trial.suggest_float('droupout_rate', 0., 0.6)

    d_model = trial.suggest_int('d_model', 20, 145) #
    num_layers = trial.suggest_int('num_layers', 1, 2)
    num_heads = trial.suggest_int('num_heads', 2, 4)
    ff_dim = trial.suggest_int('ff_dim', 32, 400)
    dropout_rate = trial.suggest_float('droupout_rate', 0., 0.28)

    with open ("./dataset complete/character_to_prediction_index.json", "r") as f:
        characters = json.load(f)

    output_vocab_size = len(characters) + 2

    #print('Got Values')

    transformer = get_compiled_transformer(
        d_model=d_model,
        num_layers=num_layers,
        num_heads=num_heads,
        ff_dim=ff_dim,
        dropout_rate=dropout_rate,
        output_vocab_size=output_vocab_size)

    #print('Got Transformer')

    callbacks = [
        optuna.integration.TFKerasPruningCallback(trial, 'val_loss'),
    ]
    # print('Training....')
    history = transformer.fit(data_gen_train, epochs=epochs, batch_size=BATCH_SIZE, validation_data=data_gen_val, callbacks=callbacks)
    #print('Training complete!')

    eval_results = transformer.evaluate(data_gen_test, batch_size = BATCH_SIZE)
    print(f'Result:{float(eval_results[0]) }')

    return float(eval_results[0])

In [34]:
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.SuccessiveHalvingPruner(), sampler=optuna.samplers.TPESampler())

#100
study.optimize(objective, n_trials=50)

pruned_trials = study.get_trials(states=[optuna.trial.TrialState.PRUNED])
complete_trials = study.get_trials(states=[optuna.trial.TrialState.COMPLETE])

print('Study statistics:')
print('   Number of finished Trials: ', len(study.trials))
print('   Number of pruned Trials: ', len(pruned_trials))
print('   Number of complete Trials: ', len(complete_trials))

print('Best Trial: ')
trial = study.best_trial

print('  Masked Loss: ', trial.value)

print('   Params: ')
for key, value in trial.params.items():
    print('   {}: {}'.format(key, value))

[I 2023-07-11 09:09:32,096] A new study created in memory with name: no-name-337031c5-7846-4e01-bbef-47da830462be


Epoch 1/5
2394/2394 [==============================] - 178s 73ms/step - loss: 2.9731 - masked_accuracy: 0.1660 - val_loss: 2.5226 - val_masked_accuracy: 0.2307
Epoch 2/5
2394/2394 [==============================] - 175s 73ms/step - loss: 2.6098 - masked_accuracy: 0.2204 - val_loss: 2.4216 - val_masked_accuracy: 0.2671
Epoch 3/5
2394/2394 [==============================] - 176s 73ms/step - loss: 2.4862 - masked_accuracy: 0.2513 - val_loss: 2.2678 - val_masked_accuracy: 0.3094
Epoch 4/5
2394/2394 [==============================] - 174s 73ms/step - loss: 2.3822 - masked_accuracy: 0.2803 - val_loss: 2.1621 - val_masked_accuracy: 0.3374
Epoch 5/5
482/482 [==============================] - 27s 57ms/step - loss: 2.0122 - masked_accuracy: 0.3842


[I 2023-07-11 09:24:37,266] Trial 0 finished with value: 2.0121541023254395 and parameters: {'d_model': 130, 'num_layers': 1, 'num_heads': 2, 'ff_dim': 381, 'droupout_rate': 0.11111424528918497}. Best is trial 0 with value: 2.0121541023254395.


Result:2.0121541023254395
Epoch 1/5
2394/2394 [==============================] - 281s 116ms/step - loss: 3.0722 - masked_accuracy: 0.1509 - val_loss: 2.6053 - val_masked_accuracy: 0.2171
Epoch 2/5
2394/2394 [==============================] - 278s 116ms/step - loss: 2.6722 - masked_accuracy: 0.2012 - val_loss: 2.5083 - val_masked_accuracy: 0.2517
Epoch 3/5
2394/2394 [==============================] - 278s 116ms/step - loss: 2.5811 - masked_accuracy: 0.2210 - val_loss: 2.3950 - val_masked_accuracy: 0.2749
Epoch 4/5
2394/2394 [==============================] - 278s 116ms/step - loss: 2.5417 - masked_accuracy: 0.2295 - val_loss: 2.3823 - val_masked_accuracy: 0.2869
Epoch 5/5
482/482 [==============================] - 25s 51ms/step - loss: 2.3174 - masked_accuracy: 0.2874


[I 2023-07-11 09:48:16,457] Trial 1 finished with value: 2.3174233436584473 and parameters: {'d_model': 40, 'num_layers': 2, 'num_heads': 4, 'ff_dim': 73, 'droupout_rate': 0.21955007933431644}. Best is trial 0 with value: 2.0121541023254395.


...

Epoch 1/5
2394/2394 [==============================] - 188s 78ms/step - loss: 2.9956 - masked_accuracy: 0.1611 - val_loss: 2.6200 - val_masked_accuracy: 0.2366
Epoch 2/5
2394/2394 [==============================] - 187s 78ms/step - loss: 2.6219 - masked_accuracy: 0.2162 - val_loss: 2.3871 - val_masked_accuracy: 0.2800
Epoch 3/5
2394/2394 [==============================] - 187s 78ms/step - loss: 2.5066 - masked_accuracy: 0.2431 - val_loss: 2.3250 - val_masked_accuracy: 0.2962
Epoch 4/5
2394/2394 [==============================] - 187s 78ms/step - loss: 2.4343 - masked_accuracy: 0.2614 - val_loss: 2.2384 - val_masked_accuracy: 0.3212
Epoch 5/5
2394/2394 [==============================] - ETA: 0s - loss: 2.3913 - masked_accuracy: 0.2746

[I 2023-07-11 16:27:48,148] Trial 48 pruned. Trial was pruned at epoch 4.


Epoch 1/5
2394/2394 [==============================] - 212s 88ms/step - loss: 2.9635 - masked_accuracy: 0.1682 - val_loss: 2.5424 - val_masked_accuracy: 0.2230
Epoch 2/5
2394/2394 [==============================] - ETA: 0s - loss: 2.6194 - masked_accuracy: 0.2199

[I 2023-07-11 16:34:51,520] Trial 49 pruned. Trial was pruned at epoch 1.


Study statistics:
   Number of finished Trials:  50
   Number of pruned Trials:  44
   Number of complete Trials:  6
Best Trial: 
  Masked Loss:  2.0121541023254395
   Params: 
   d_model: 130
   num_layers: 1
   num_heads: 2
   ff_dim: 381
   droupout_rate: 0.11111424528918497
